## VacationPy

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [ ]:
# Read the csv from WeatherPy
weather_csv = pd.read_csv("../WeatherPy/Output/cities.csv")
weather_csv = weather_csv.drop(columns= ["Unnamed: 0"])
weather_csv.head()

## Humidity Heatmap

In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [ ]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_csv[["Lat", "Lng"]]

humidity = weather_csv["Humidity"]

In [ ]:
# Create a humidity Heatmap layer
weather_heatmap = gmaps.figure(center=(0,0), zoom_level = 1.5)

humidity_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating= False, max_intensity=100,
                                 point_radius = 1)

weather_heatmap.add_layer(humidity_layer)

weather_heatmap

## Ideal Vacation Weather

In [ ]:
weather_csv = weather_csv.loc[weather_csv["Max Temp"] <= 80, :]
weather_csv = weather_csv.loc[weather_csv["Max Temp"] >= 70, :]
weather_csv = weather_csv.loc[weather_csv["Wind Speed"] <= 6.8, :]
weather_csv = weather_csv.loc[weather_csv["Cloudiness"] == 0, :]
weather_csv

## Hotel Map

In [ ]:
hotel_df = weather_csv
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# params dictionary to update each iteration
params = {"radius": 50000,
          "keyword": "hotel",
          "key": g_key}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    nearest_hotels = requests.get(base_url, params=params)

    # convert to json
    nearest_hotels = nearest_hotels.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = nearest_hotels["results"][0]["name"]
        print("Hotel found")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

weather_heatmap.add_layer(hotel_layer)

# Display figure
weather_heatmap